In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [2]:
import warnings
warnings.simplefilter('ignore')

In [3]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

### Read the dataset for churn 

In [4]:
df = pd.read_csv("new_variant_1.csv")

# Model tuning


#### Parameters vs Hyperparameters
Параметр можна вважати внутрішнім або інтерним до моделі, який можна отримати після того, як модель навчилася на даних. Прикладами параметрів є коефіцієнти регресії в лінійній регресії, опорні вектори в методах опорних векторів і ваги в нейронних мережах.

Гіперпараметр можна вважати зовнішнім або екстраним до моделі, який може бути заданий довільно практиком. Прикладами гіперпараметрів є значення k в k-найближчих сусідах, кількість дерев і максимальна кількість ознак у випадковому лісі, швидкість навчання та моментум у нейронних мережах, параметри C і gamma в методах опорних векторів.

#### Налаштування гіперпараметрів
Оскільки немає універсально найкращих гіперпараметрів для будь-якої проблеми, гіперпараметри зазвичай встановлюються на значення за замовчуванням. Однак оптимальний набір гіперпараметрів можна отримати за допомогою ручного емпіричного (методом спроб і помилок) пошуку гіперпараметрів або в автоматичний спосіб за допомогою оптимізаційного алгоритму для максимізації функції придатності.

Два поширені методи налаштування гіперпараметрів включають grid search (пошук по сітці) та random search (випадковий пошук). Як випливає з назви, пошук по сітці передбачає створення сітки можливих значень гіперпараметрів, для яких моделі поступово будуються для всіх комбінацій цих гіперпараметрів у брутальному режимі. У випадковому пошуку не всі комбінації гіперпараметрів використовуються, але замість цього на кожній ітерації використовується випадкова комбінація гіперпараметрів.

#### Define target and features columns

In [5]:
y_column = ['Status'] # target variable
X_columns = ['loan_limit', 'Gender', 'approv_in_adv', 'loan_type', 'loan_purpose', 'Credit_Worthiness', 'open_credit', 
             'business_or_commercial', 'loan_amount', 'rate_of_interest', 'Interest_rate_spread', 'Upfront_charges', 'term', 
             'Neg_ammortization', 'interest_only', 'lump_sum_payment', 'property_value', 'construction_type', 
             'occupancy_type', 'Secured_by', 'total_units', 'income', 'credit_type', 'Credit_Score', 'co-applicant_credit_type', 
             'age', 'submission_of_application', 'LTV', 'Region', 'Security_Type',]

X = df[X_columns]
y = df[y_column]

In [6]:
# Let's say we want to split the data in 80:10:10 for train:valid:test dataset
# In the first step we will split the data in training and remaining dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(115669, 30)
(115669, 1)
(28918, 30)
(28918, 1)


#### Building a Baseline LogisticRegression Model

In [7]:
lr = LogisticRegression(class_weight='balanced')

In [8]:
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [9]:
print('test set metrics: \n', metrics.classification_report(y_test, y_pred))

test set metrics: 
               precision    recall  f1-score   support

          no       0.86      0.66      0.74     21887
         yes       0.38      0.66      0.48      7031

    accuracy                           0.66     28918
   macro avg       0.62      0.66      0.61     28918
weighted avg       0.74      0.66      0.68     28918



In [10]:
lr.get_params()

{'C': 1.0,
 'class_weight': 'balanced',
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'deprecated',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

#### Hyperparameter Tuning
Now we will be performing the tuning of hyperparameters of the LogisticRegression model. 

In [13]:
%%time
# Create the random grid
param_grid = {
    'penalty': ['l1', ],
    'C': [0.1, 1],
    'solver': ['liblinear', 'saga'],
    'max_iter': [100, 200,],
    'class_weight': ['balanced'],
}


lr = LogisticRegression(class_weight='balanced')

grid = GridSearchCV(estimator=lr, 
                    param_grid=param_grid, 
                    scoring='accuracy', 
                    cv=5,
                    verbose=3,
                    return_train_score=True)

grid.fit(X_train, y_train)

print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END C=0.1, class_weight=balanced, max_iter=100, penalty=l1, solver=liblinear;, score=(train=0.688, test=0.686) total time=  44.6s
[CV 2/5] END C=0.1, class_weight=balanced, max_iter=100, penalty=l1, solver=liblinear;, score=(train=0.688, test=0.686) total time=  26.7s
[CV 3/5] END C=0.1, class_weight=balanced, max_iter=100, penalty=l1, solver=liblinear;, score=(train=0.689, test=0.685) total time=  21.0s
[CV 4/5] END C=0.1, class_weight=balanced, max_iter=100, penalty=l1, solver=liblinear;, score=(train=0.687, test=0.690) total time=  13.5s
[CV 5/5] END C=0.1, class_weight=balanced, max_iter=100, penalty=l1, solver=liblinear;, score=(train=0.688, test=0.692) total time=  26.5s
[CV 1/5] END C=0.1, class_weight=balanced, max_iter=100, penalty=l1, solver=saga;, score=(train=0.653, test=0.645) total time=   5.7s
[CV 2/5] END C=0.1, class_weight=balanced, max_iter=100, penalty=l1, solver=saga;, score=(train=0.651, test=0.6

In [14]:
grid_results = pd.concat([pd.DataFrame(grid.cv_results_["params"]),
                          pd.DataFrame(grid.cv_results_["mean_test_score"], 
                          columns=["precision"])],
                          axis=1)

grid_results

,C,class_weight,max_iter,penalty,solver,precision
0,0.1,balanced,100,l1,liblinear,0.687851
1,0.1,balanced,100,l1,saga,0.651151
2,0.1,balanced,200,l1,liblinear,0.688050
3,0.1,balanced,200,l1,saga,0.663523
4,1.0,balanced,100,l1,liblinear,0.688136
5,1.0,balanced,100,l1,saga,0.650944
6,1.0,balanced,200,l1,liblinear,0.688145
7,1.0,balanced,200,l1,saga,0.663912


In [15]:
grid_results.columns

Index(['C', 'class_weight', 'max_iter', 'penalty', 'solver', 'precision'], dtype='object')